In [3]:
import numpy as np
from src import waveforms as wf, fisher_matrix as fm, detectors as det
import lalsimulation as ls
from astropy.cosmology import Planck15 as cosmo
from scipy.interpolate import interp1d

In [14]:
## First the source properties
m1,m2 = 30,20
z = 0.6

## Then the detector properties
psd = det.load_detector_psd(detector='LIGO_Design')
fraction_detectable = det.load_cumulative_fraction(network='HLV')
snr_threshold = 8

## Set binary properties for waveform generator
mtot = m1+m2
q = np.maximum(m1,m2)/np.minimum(m1,m2) ## NOTE: q > 1
dist = cosmo.luminosity_distance(z).value

## Wrapper for waveform generator
hp,hx,freqs = wf.hp_hx_inspiral(mtot,q,dist,approx=ls.IMRPhenomD,df=0.1)

## Compute optimal SNR and fraction of sources above threshold
snr = fm.compute_SNR(hp,freqs,psd)
detectable = fraction_detectable(snr_threshold/snr)

## finally compute the weight, multiplying the fraction of detectable sources
## by dV/dz at redshift z, divided by 1+z (merger rate at source vs detector)
weight = detectable*cosmo.differential_comoving_volume(z).value*4*np.pi/(1+z)